# Mini-lab 3 - Geocoding, Maps, and Markers

In [1]:
from datascience import *
import numpy as np
from geopy.geocoders import Nominatim

### Geocoding
Geocoding is the process of converting locations (such as address or points of interest) into geographic coordinates (such as lat, lon coordinates). The geopy.geocoders package gives us access to several geolocation API's (like google maps, Nominatim - open street maps API, bing maps API, etc.). The documentations for the geocoding package can be found here: https://geopy.readthedocs.io/en/1.10.0/. 


#### Geocoding example
Below is an example of how the geocoding service works:

In [2]:
geolocator = Nominatim()
location = geolocator.geocode("175 5th Avenue NYC")
location


Location(Flatiron Building, 175, 5th Avenue, Flatiron District, Manhattan, Manhattan Community Board 5, New York County, NYC, New York, 10010, USA, (40.7410861, -73.9896298241625, 0.0))

#### location properties
The geolocator.geocode() returns a Location object. The location objects have the following properites:

In [3]:
print (location.latitude)
print (location.longitude)
print (location.altitude) 
print (location.address)

40.7410861
-73.9896298241625
0.0
Flatiron Building, 175, 5th Avenue, Flatiron District, Manhattan, Manhattan Community Board 5, New York County, NYC, New York, 10010, USA


#### Use the geolocater to find the lat, lon coordinates for the Statue of Liberty

In [4]:
liberty = geolocator.geocode("Statue of Liberty")
lib_lat = liberty.latitude
lib_lon = liberty.longitude
liberty

Location(Statue of Liberty, Flagpole Plaza, Manhattan Community Board 1, New York County, NYC, Liberty Island, 10004, USA, (40.6892474, -74.0445405280149, 0.0))

# Creating maps
If we generate a table with columns 'latitude', 'longitude', 'popup', and 'color', then we can generate maps with  markers from the table using the maps.Marker.map_table() method. See the example below. Click on the marker and the popup text should appear. 

In [5]:
location = geolocator.geocode("175 5th Avenue NYC")
table = Table().with_columns(['latitude', [location.latitude],
                              'longitude', [location.longitude],
                              'popup', [location.address],
                              'color', ['red']])
table

latitude,longitude,popup,color
40.7411,-73.9896,"Flatiron Building, 175, 5th Avenue, Flatiron District, M ...",red


In [6]:
maps.Marker.map_table(table)

/srv/app/venv/lib/python3.6/site-packages/datascience/maps.py:368: FutureWarning: simple_marker is deprecated. Use add_children(Marker) instead
  f(**self._folium_kwargs)


### Add a row to the map table for the Statue of Liberty. Make the color 'green'.
The [Table.append()](http://data8.org/datascience/_autosummary/datascience.tables.Table.append.html?highlight=append#datascience.tables.Table.append) method may come in handy for adding a row.

In [7]:
table = table.append([lib_lat, lib_lon, liberty.address, "green"])
table

latitude,longitude,popup,color
40.7411,-73.9896,"Flatiron Building, 175, 5th Avenue, Flatiron District, M ...",red
40.6892,-74.0445,"Statue of Liberty, Flagpole Plaza, Manhattan Community B ...",green


In [8]:
# after you have added the row, use the following to map the table
maps.Marker.map_table(table)

/srv/app/venv/lib/python3.6/site-packages/datascience/maps.py:368: FutureWarning: simple_marker is deprecated. Use add_children(Marker) instead
  f(**self._folium_kwargs)


### Variations in map style:
If we add a radius column to our table, we can use circle markers instead of info markers. See the example below

In [19]:
table['radius'] = 200000
maps.Circle.map_table(table)

/srv/app/venv/lib/python3.6/site-packages/datascience/maps.py:368: FutureWarning: circle_marker is deprecated. Use add_children(CircleMarker) instead
  f(**self._folium_kwargs)


### Mapping utilities and services in San Francisco
Mapping can be an important tool for visualizing spatial data. In the rest of this lab we will become more familiar with the mapping tools.

#### Firestations in SF
'firestations.csv' contains a list of addresses of firestations in San Francisco. 

In [10]:
firestations = Table.read_table('firestations.csv')
firestations

Station #,Address
Station 1,935 Folsom Street
Station 2,1340 Powell Street
Station 3,1067 Post Street
Station 4,449 Mission Rock Street
Station 5,1301 Turk Street
Station 6,135 Sanchez Street
Station 7,2300 Folsom Street
Station 8,36 Bluxome Street
Station 9,2245 Jerrold Avenue
Station 10,655 Presidio Avenue


**Your tasks:** 
1. Get the first address from the firestation table. Add ", San Francisco, CA" to this address.
1. Use the geolocater to find the lat, lon coordinates of this address.


In [11]:
first_add = firestations.column(1).item(0) + ", San Francisco, CA"
first_add_geo = geolocator.geocode(first_add)
first_add_geo

Location(SFFD Station 1, 935, Folsom Street, South of Market, SF, California, 94103, USA, (37.77941665, -122.404100451486, 0.0))

### Mapping all firestations
1. Use a **for loop** to iterate through all addresses in the dataframe. Add ", San Francisco, CA" to each address, then use the geolocater to find the lat, lon coordinates of the address,  
1. Append the latitude and longitude info to the firestations_map_table. Set the popup text = the Station # (from the firestation table), make the color "black" and the radius 200. 
1. Map the firestation_map_table using "maps.Circle.map_table()" method


In [12]:
#Your code here
fire_stations_map_table = Table(['latitude', 'longitude', 'popup', 'color', 'radius'])
fire_geo = []
for address in firestations.column(1):
    station = firestations.where(1, address).column(0).item(0)
    new_loc = geolocator.geocode([address + ", San Francisco, CA"])
    fire_stations_map_table.append([new_loc.latitude, new_loc.longitude, station, "black", 200000])
maps.Circle.map_table(fire_stations_map_table)

/srv/app/venv/lib/python3.6/site-packages/datascience/maps.py:368: FutureWarning: circle_marker is deprecated. Use add_children(CircleMarker) instead
  f(**self._folium_kwargs)


### Bonus problem: There's a fire at City Hall!
If you want an extra challenge:

There's a fire at City Hall! We need to send firetrucks from the closest 3 firestations ASAP! Find City Hall, then find the 3 closest firestations and update your map by coloring these firestations red.
1. Add a large blue marker to the firestaion_map_table for the city hall.
1. Locate the closest 3 firestations to SF city hall 
1. Change the color of these firestation to red in your fire_stations_map_table.

In [13]:
city_hall = geolocator.geocode("San Francisco City Hall")
fire_stations_map_table.append([city_hall.latitude, city_hall.longitude, "City Hall", "blue", 200000])
#maps.Circle.map_table(fire_stations_map_table)

latitude,longitude,popup,color,radius
37.7794,-122.404,Station 1,black,200000
37.797,-122.41,Station 2,black,200000
37.7866,-122.419,Station 3,black,200000
37.7728,-122.389,Station 4,black,200000
37.782,-122.421,Station 5,black,200000
37.7671,-122.431,Station 6,black,200000
37.7601,-122.415,Station 7,black,200000
37.7773,-122.397,Station 8,black,200000
37.7452,-122.401,Station 9,black,200000
37.7856,-122.447,Station 10,black,200000


In [14]:
def distance(lat, lon):
    return (lat-city_hall.latitude)**2 + (lon-city_hall.longitude)**2

closest_fire_stations = fire_stations_map_table.with_column(
    "distance", fire_stations_map_table.apply(distance, "latitude", "longitude")
).sort("distance").take(np.arange(4)).exclude(0)
closest_fire_stations_names = closest_fire_stations.column("popup")
closest_fire_stations_names

array([' Station 5', 'Station 36', 'Station 3'], dtype='<U10')

In [16]:
#def change(t, col_label, change):
#    """takes in a table t, a column label col_label, and the change value, \
#    and return a table with selected column swapped out for new values"""
#    column = t.column_index(col_label)
#    moved = t.select(np.arange(column+1, t.num_cols-1))
#   keep = t.drop(moved).drop(t.num_cols-1)
#    new = keep.with_columns(col_label, change).with_columns
    
fire = fire_stations_map_table.where(2, are.contained_in(closest_fire_stations_names)).with_columns("color", "red")
fire_stations_map_table = fire_stations_map_table.exclude([2,4,35]).append(fire)

In [17]:
fire_stations_map_table.show()

latitude,longitude,popup,color,radius
37.7794,-122.404,Station 1,black,200000
37.797,-122.41,Station 2,black,200000
37.7728,-122.389,Station 4,black,200000
37.7671,-122.431,Station 6,black,200000
37.7601,-122.415,Station 7,black,200000
37.7773,-122.397,Station 8,black,200000
37.7452,-122.401,Station 9,black,200000
37.7856,-122.447,Station 10,black,200000
37.7488,-122.426,Station 11,black,200000
37.7635,-122.453,Station 12,black,200000


In [18]:
maps.Circle.map_table(fire_stations_map_table)

/srv/app/venv/lib/python3.6/site-packages/datascience/maps.py:368: FutureWarning: circle_marker is deprecated. Use add_children(CircleMarker) instead
  f(**self._folium_kwargs)
